In [31]:
!pip install cassandra-driver -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 984.2 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: cassandra-driver
    Found existing installation: cassandra-driver 3.28.0
    Uninstalling cassandra-driver-3.28.0:
      Successfully uninstalled cassandra-driver-3.28.0

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
from dotenv import load_dotenv
from conn import getCQLSession
from cassandra.cqlengine.query import BatchStatement
from cassandra.query import BatchType
import csv

load_dotenv( override=True) 
os.environ["MODE"]

'astra_db'

In [7]:
from conn import getCQLSession
session = getCQLSession(os.environ["MODE"])

Initializing CQL Session
Connected


In [8]:
table = "default_keyspace.product_catalog_emb"

session.execute("""CREATE TABLE IF NOT EXISTS default_keyspace.product_catalog_emb (
                    id text PRIMARY KEY,
                    title text,
                    emb VECTOR<FLOAT, 384>)""")


#First index model
stmt = """CREATE CUSTOM INDEX product_catalog_emb_vector_idx
    ON default_keyspace.product_catalog_emb (emb)
    USING 'StorageAttachedIndex'  
    WITH OPTIONS = { 'similarity_function': 'COSINE'}""""

#considering quantization for BERT
stmt = """CREATE CUSTOM INDEX product_catalog_emb_vector_idx
    ON default_keyspace.product_catalog_emb (emb)
    USING 'StorageAttachedIndex'  
    WITH OPTIONS = { 'source_model': 'bert', 'similarity_function': 'COSINE'}"""
    
#considering quantization for BERT and with DOT_Product
stmt = """CREATE CUSTOM INDEX product_catalog_emb_vector_idx
    ON default_keyspace.product_catalog_emb (emb)
    USING 'StorageAttachedIndex'  
    WITH OPTIONS = { 'source_model': 'bert', 'similarity_function': 'DOT_PRODUCT'};"""  

session.execute(stmt)


InvalidRequest: Error from server: code=2200 [Invalid query] message="Index 'product_catalog_emb_vector_idx' already exists"

In [15]:
cmd_insert = f"""
INSERT INTO {table} (id, title, emb)
VALUES (:id, :title, :emb)
"""

prepared_stmt_insert = session.prepare(cmd_insert)

In [10]:
def read_csv(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            data.append(row)
    return data

In [39]:
def save_data(data, batch_size = 20):
    batch = BatchStatement(batch_type=BatchType.UNLOGGED)
    count = 0
    for row in data:
        batch.add(prepared_stmt_insert, {"id": row['asin'], "title": row['title'], "emb": [float(x) for x in row['emb'].split(',')] })
        count += 1
        if count % batch_size == 0:
            rs = session.execute(batch)
            batch.clear()

        if count % 1000 == 0 :
            print(f"""{count} records inserted.""")
    
    
    rs = session.execute(batch)
    batch.clear()
    
    return count

In [12]:
data = read_csv("../data_embeddings/chunk_9999.csv")

In [29]:
len(data)

10000

In [27]:
len([float(x) for x in data[0]['emb'].split(',')])

384

In [53]:
import time
start = time.time()
save_data(data,100)
end = time.time()
print(end - start)

1000 records inserted.
2000 records inserted.
3000 records inserted.
4000 records inserted.
5000 records inserted.
6000 records inserted.
7000 records inserted.
8000 records inserted.
9000 records inserted.
10000 records inserted.
77.29538297653198
